# Decorators properly explained!

Do you have trouble understanding decorators? You're not alone. I've identified a couple of mental hurdles that prevent beginners from completely wrapping their head around decorators, even though they may already be using them. My goal is to break down these hurdles, so that you can write your own decorators and live happily ever after! 

## Why decorators?

Decorators are a really nice way to implement seperation of concerns: the idea that a function should do only one thing. Seperation of concerns produces cleaner code. That's by no means the only use-case for decorators, but it's a major one. I have neither the patience nor the knowledge to describe them all. 

## A quick example

Say you have a function that substracts 2 from its input. On top of that, you wanna write to the log every time the function is called. You could add a line inside your function telling it to write to the log file, but that's breaking the rule that a function should do only one thing. 

Decorator to the rescue! First write a logger decorator, then apply it to your function, then just call the function like normal... et voila... you have seperation of concerns! The big bonus is that you're free to reuse that decorator as much as you want on other functions. Tada! You also have dry code.

Ok, so with python's decorator syntax, what I've just described looks like this:

In [1]:
def log_me(function):
    print('%s is calculating!' % function)
    return function

@log_me
def minus_two(n):
    return n - 2

addition = minus_two(7)
addition

<function minus_two at 0x7f6515df6f28> is calculating!


5

## Decorator definition

In the above example, `log_me` is the decorator and `minus_two` is being decorated. And now comes the take away point: __a decorator is a function that returns a function__. Check it out... that's precisely what `log_me` does. All decorators must return a function. In fact, the above script is pure __syntactic sugar__ and is equivalent to:

In [2]:
addition = log_me(minus_two)(7)
addition

<function minus_two at 0x7f6515df6f28> is calculating!


5

Now we're ready to roll. We'll first learn to monkey-patch *without* the specific decorator syntax. Once that's understood, we'll move on to the sweeter stuff. So... let's go ahead and monkey-patch one step at a time!...

## Step 1. A simple identity decorator 
Okay, so in this tutorial we'll be monkey-patching functions, meaning __substituting functions on the fly__. This is all you'll need to know. Let's start with the simplest monkey-patch: the identity function. The identity function is a function that returns its input without changing it.

In [3]:
def identity(f):
    print('Inside identity I could monkeypatch %s if I wanted to!' % f.__name__)
    return f

the_same = identity(minus_two)
the_same    

Inside identity I could monkeypatch minus_two if I wanted to!


<function __main__.minus_two>

Now let's add a message inside `minus_two` itself, so we can follow the code sequence. It might look trivial but it will come in handy later, when things start to get more complicated.


In [4]:
def minus_two(n):
    print('Now inside minus_two with n = %s' % n)
    return n - 2

ten = minus_two(12)
ten

Now inside minus_two with n = 12


10

Nothing's changed except our little function is now a little more verbose. Okay, it's time to apply the `identity` function to `minus_two`. We *should* get the same result... drum roll...

In [5]:
result = identity(minus_two)(12)
result

Inside identity I could monkeypatch minus_two if I wanted to!
Now inside minus_two with n = 12


10

## Step 2. A real monkey-patch
That wasn't very interesting, let's *really* monkey-patch the function `minus_two`, say with a function called `plus_four`. Out `monkey_patch` function will take any function object and always return the input value + 4! Not rocket science by any stretch of the imagination.

In [6]:
def plus_four(n):
    print('Now inside the function plus_four with n = %s' % n)
    return n + 4

def monkey_patch(f):
    print('Inside monkey-patch I swap %s with %s' % (f.__name__, plus_four.__name__))
    return plus_four

not_the_same = monkey_patch(minus_two)
not_the_same

Inside monkey-patch I swap minus_two with plus_four


<function __main__.plus_four>

We have our monkey-patch decorator function! Now let's apply it to the good old function `minus_two`...

In [7]:
result = monkey_patch(minus_two)(18)
result

Inside monkey-patch I swap minus_two with plus_four
Now inside the function plus_four with n = 18


22

## Step 3. Python's syntactic sugar
That's the whole point. Python provides a nice expressive and concise way of achieving exactly what we've just done! So let's use it...

In [8]:
@monkey_patch
def minus_two(n):
    print('Now I am inside the function minus_two with n = %s' % n)
    return n - 2

result = minus_two(1)
result

Inside monkey-patch I swap minus_two with plus_four
Now inside the function plus_four with n = 1


5

Cool. Get this: from the perspective of the coder calling the function `minus_two`, we've changed the rules of algebra! From now on 1 - 2 = 5. Yeah.

## Step 4. Scopes 101
Before we can go any further, we need to take a step back for a second and turn our attention to scopes. In python, functions (as well as methods and classes) always have access to their outer scope, like so:

In [9]:
a = 'hello from the outer scope'

def print_a_variable_from_the_outer_scope():
    print(a)
    
print_a_variable_from_the_outer_scope()

hello from the outer scope


This is something we use all the time without noticing. For example, when we import a class or function or any object into a python module, it's available everywhere in the module thereafter. The failure to recognize this is often the reason why beginners fail to understand decorator logic. That included me. I had that eureka moment. Let me share it.

## Step 5. A parameterizable decorator
Let's go one step further and write a decorator that replaces a function by an another one, but not always the same one. We want to tell a generic decorator which function to substitute. To achieve that, we'll need the concept of scopes we've just explained. Ok, so first, let's volunteer two substitute functions we'll be able to choose from: `divide_by_six` and `multiply_by_twelve`.

In [17]:
def divide_by_six(n):
    print('Inside the function divide_by_six with n = %s' % n)
    return n / 6

def times_twelve(n):
    print('Inside the function multiply_by_twelve with n = %s' % n)
    return n * 12

Take a look at the following script. In a nutshell, what we want to do is write the decorator `replace_with` to make the script work. For now, it's it returns f and the script gives an error. What's going on? 

In [18]:
def replace_with(f):
    # f is the function we want to subsitute. 
    return f

@replace_with(times_twelve)
def minus_five(n):
    print('Now I am inside the function minus_five with n = %s' % n)
    return n - 5


Inside the function multiply_by_twelve with n = <function minus_five at 0x7f651514d598>


TypeError: unsupported operand type(s) for *: 'function' and 'int'

Hint. Remember that a decorator is a function that returns another function. Really? Actually, no. I lied to you: as a matter of fact, it's the __entire expression after the `@` sign__ that must return a function! This subtelty is usually another gotcha for beginners. Hang on tight. It means that the function `replace_with` must itself __return a function that returns a function__. Hang on tight, like I said!... here's the way to do it:  

In [19]:
def replace_with(substitute):
    print('The substitute function is %s' % substitute.__name__)
    def calculator(original):
        print('Inside the wrapper function I can use %s from the outer scope' % substitute.__name__)
        return substitute
    return calculator

Indeed, in the above, `replace_with` returns the function `calculator`, a wrapper function that returns a function. That was the first trick. The second trick was: `calculator` had access to its outer scope and that's where it got the `substitute` variable from. Basically, if you get this, you've understood the basic mechanism behind decorators. Nuff said. It's time to apply our decorator. 

In [20]:
@replace_with(divide_by_six)
def minus_five(n):
    print('Now I am inside the function minus_five with n = %s' % n)
    return n - 5

high_jacked = minus_five(10)
high_jacked

The substitute function is divide_by_six
Inside the wrapper function I can use divide_by_six from the outer scope
Inside the function divide_by_six with n = 10


1.6666666666666667

... and again for good measure...

In [21]:
@replace_with(times_twelve)
def minus_five(n):
    print('Now I am inside the function minus_five with n = %s' % n)
    return n - 5

high_jacked = minus_five(10)
high_jacked

The substitute function is times_twelve
Inside the wrapper function I can use times_twelve from the outer scope
Inside the function multiply_by_twelve with n = 10


120

## Conclusion

Obviously, there's a lot more to decorators than what we've covered in this tutorial. But we'll stop here for now. The goal was to overcome a couple of mental blockages people typically have when they first come across the concept. I would like to write more on the subject. The next logical step would be to show how you can make decorators with class instances or even classes themselves. And also how you can decorate not only functions, but classes too. Anyways, that's it for now! Ciao ciao.